In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import prince
from prince import FAMD
import scipy.spatial.distance as dist

# 2.1 Getting your data 

We want to read the datasets to try to understand which key we can use to merge them together.

First of all we want to fill empty values in the datasets.
The below function take as an input a dataframe and check if in the columns there are any missing values of numeric or string types and fills this value with an empty string, in case of strings, or with a mean of the values of that column in case of numbers.

In [109]:
def filler(dataframe):
    for col in dataframe.columns:
        if dataframe[col].isnull().any():
            if is_string_dtype(dataframe[col]):
                dataframe[col] = dataframe[col].fillna("")
            elif is_numeric_dtype(dataframe[col]):
                dataframe[col] = dataframe[col].fillna(dataframe[col].mean())

In [110]:
tracks = pd.read_csv("tracks.csv")
echonest = pd.read_csv("echonest.csv")
features = pd.read_csv("features.csv")

We take the datasets needed:

In [69]:
tracks.head()

,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,...,track_information,track_interest,track_language_code,track_license,track_listens,track_lyricist,track_number,track_publisher,track_tags,track_title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,NaN,3,NaN,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,NaN,4,NaN,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,NaN,6,NaN,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4,6,NaN,47632,NaN,...,NaN,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,NaN,1,NaN,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,NaN,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,NaN,3,NaN,[],Spiritual Level


In [70]:
tracks.dtypes

track_id                      int64
album_comments                int64
album_date_created           object
album_date_released          object
album_engineer               object
album_favorites               int64
album_id                      int64
album_information            object
album_listens                 int64
album_producer               object
album_tags                   object
album_title                  object
album_tracks                  int64
album_type                   object
artist_active_year_begin     object
artist_active_year_end       object
artist_associated_labels     object
artist_bio                   object
artist_comments               int64
artist_date_created          object
artist_favorites              int64
artist_id                     int64
artist_latitude             float64
artist_location              object
artist_longitude            float64
artist_members               object
artist_name                  object
artist_related_projects     

In [71]:
tracks.shape

(106574, 53)

In [72]:
echonest.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,...,temporal_features_214,temporal_features_215,temporal_features_216,temporal_features_217,temporal_features_218,temporal_features_219,temporal_features_220,temporal_features_221,temporal_features_222,temporal_features_223
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,NaN,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,NaN,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,NaN,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,NaN,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705


In [73]:
echonest.shape

(13129, 250)

In [74]:
features.head()

,track_id,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [75]:
features.shape

(106574, 519)

(note that: features and tracks have the same number of rows)

We can notice that a common column is **track_id**, so let's check if there are any missing values:

In [76]:
tracks["track_id"].isnull().any()

False

In [77]:
echonest["track_id"].isnull().any()

False

In [78]:
features["track_id"].isnull().any()

False

Given that there are no missing values in "**track_id**" columns in any dataset, we choose to use it as the key for the merge of the datasets.

We build a copy and work on it to don't damage the original files.

In [111]:
tracks_c = tracks.copy()
echonest_c = echonest.copy()
features_c = features.copy()

In [112]:
# First we are gonna fill the datasets using the function filler()
filler(tracks_c)
filler(echonest_c)
filler(features_c)

In [113]:
tracks_c.head()

,track_id,album_comments,album_date_created,album_date_released,album_engineer,album_favorites,album_id,album_information,album_listens,album_producer,...,track_information,track_interest,track_language_code,track_license,track_listens,track_lyricist,track_number,track_publisher,track_tags,track_title
0,2,0,2008-11-26 01:44:45,2009-01-05 00:00:00,,4,1,<p></p>,6073,,...,,4656,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293,,3,,[],Food
1,3,0,2008-11-26 01:44:45,2009-01-05 00:00:00,,4,1,<p></p>,6073,,...,,1470,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514,,4,,[],Electric Ave
2,5,0,2008-11-26 01:44:45,2009-01-05 00:00:00,,4,1,<p></p>,6073,,...,,1933,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151,,6,,[],This World
3,10,0,2008-11-26 01:45:08,2008-02-06 00:00:00,,4,6,,47632,,...,,54881,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135,,1,,[],Freeway
4,20,0,2008-11-26 01:45:05,2009-01-06 00:00:00,,2,4,"<p> ""spiritual songs"" from Nicky Cook</p>",2710,,...,,978,en,Attribution-NonCommercial-NoDerivatives (aka M...,361,,3,,[],Spiritual Level


In [82]:
echonest_c.head()

,track_id,audio_features_acousticness,audio_features_danceability,audio_features_energy,audio_features_instrumentalness,audio_features_liveness,audio_features_speechiness,audio_features_tempo,audio_features_valence,metadata_album_date,...,temporal_features_214,temporal_features_215,temporal_features_216,temporal_features_217,temporal_features_218,temporal_features_219,temporal_features_220,temporal_features_221,temporal_features_222,temporal_features_223
0,2,0.416675,0.675894,0.634476,0.010628,0.177647,0.159310,165.922,0.576661,,...,-1.992303,6.805694,0.233070,0.192880,0.027455,0.06408,3.67696,3.61288,13.316690,262.929749
1,3,0.374408,0.528643,0.817461,0.001851,0.105880,0.461818,126.957,0.269240,,...,-1.582331,8.889308,0.258464,0.220905,0.081368,0.06413,6.08277,6.01864,16.673548,325.581085
2,5,0.043567,0.745566,0.701470,0.000697,0.373143,0.124595,100.260,0.621661,,...,-2.288358,11.527109,0.256821,0.237820,0.060122,0.06014,5.92649,5.86635,16.013849,356.755737
3,10,0.951670,0.658179,0.924525,0.965427,0.115474,0.032985,111.562,0.963590,2008-03-11,...,-3.662988,21.508228,0.283352,0.267070,0.125704,0.08082,8.41401,8.33319,21.317064,483.403809
4,134,0.452217,0.513238,0.560410,0.019443,0.096567,0.525519,114.290,0.894072,,...,-1.452696,2.356398,0.234686,0.199550,0.149332,0.06440,11.26707,11.20267,26.454180,751.147705


In [83]:
features_c.head()

,track_id,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


## 2.2 Choose your features (variables)!

We notice that the number of features is very big (820 columns!) and we want to reduce the number of variables we have to work with.

So let's apply dimensionality reduction!

###  Principal Component Analysis

Principal component analysis, is a statistical technique to convert high dimensional data to low dimensional data by selecting the most important features that capture maximum information about the dataset. The features are selected on the basis of variance that they cause in the output. The feature that causes highest variance is the first principal component. The feature that is responsible for second highest variance is considered the second principal component, and so on. It is important to mention that principal components do not have any correlation with each other.

#### Standardizing the Numeric Features

Since PCA is effected by scale, we need to scale the features in our data before applying PCA. We used StandardScaler to standardize the dataset’s features onto unit scale (mean = 0 and variance = 1) which is a requirement for the optimal performance of algorithms.First we have to chose numeric and categorical columns.

### Echonest 

In [114]:
# First we need to chose numeric and categorical columns 
num_cols = [col for col in echonest_c.columns if echonest_c[col].dtypes != "O"]
cat_cols = [col for col in echonest_c.columns if echonest_c[col].dtypes == "O"]

We decided to reduce the number of temporal features columns from 224 to 32. The reason why we did not choose it lower than this number (32) is to ensure that the total variance is greater than 70%.

In [115]:
filter_temp_col = [col for col in num_cols if col.startswith('temporal')]
#filter_temp_col

In [117]:
# We used PCA for our numerical temporal features columns
x1 = echonest_c.loc[:, filter_temp_col].values
x1 = StandardScaler().fit_transform(x1)

In [118]:
# Initialize the PCA class by passing the number of components to the constructor.
pca = PCA(n_components=32)
# Call the fit and then transform methods by passing the feature set to these methods. 
# The transform method returns the specified number of principal components.
principalComponents = pca.fit_transform(x1)
# Creating Principal data frame
principalDf_1 = pd.DataFrame(data = principalComponents
             , columns = ["principal component Echonest " + str(i) for i in range(1,33)])

In [119]:
#pca.explained_variance_ratio_
print ( "Components = ", pca.n_components_ , ";\nTotal explained variance = ",
      round(pca.explained_variance_ratio_.sum(),5)  )

Components =  32 ;
Total explained variance =  0.71034


We reduced our total column count from 250 to 58. We chose PCA which we know that it works great on continuous data, to do dimensionality reduction on the Temporal Features columns and became sure that the chosen method retains > 70% of the total variance.

In [120]:
other_num_cols = [ele for ele in num_cols if ele not in filter_temp_col]
col_list = cat_cols + other_num_cols
final_echonest = pd.concat([principalDf_1, echonest_c[col_list]], axis = 1)

In [121]:
final_echonest.shape

(13129, 58)

### Tracks

PCA works  on continuous data but our data is a blend of both continuous data and categorical data in this file. Therefore, we searched the other methods. FAMD does the analysis with a combination of PCA and MCA techniques. MCA stands for Multiple Correspondence Analysis which is suitable for multiple categorical factors specifically. If the dataset is grouped by different features with a blend of continuous and categorical values, another technique named MFA (Multiple Factor Analysis) can be deployed.

When we try to apply FAMD, MFA, and MCA for the columns in tracks, we couldn't get any acceptable results. That's why we decided to keep only useful columns in our dataset.

We decide to keep only useful variables:

In [150]:
final_tracks = tracks_c[['track_id', 'album_id', 'artist_id',"artist_location", 'track_bit_rate', 
                    'track_duration', 'track_listens',"track_language_code","track_favorites","track_genre_top",
                     "track_genres","track_genres_all","track_tags","track_title"]]

In [151]:
final_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106574 entries, 0 to 106573
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   track_id             106574 non-null  int64 
 1   album_id             106574 non-null  int64 
 2   artist_id            106574 non-null  int64 
 3   artist_location      106574 non-null  object
 4   track_bit_rate       106574 non-null  int64 
 5   track_duration       106574 non-null  int64 
 6   track_listens        106574 non-null  int64 
 7   track_language_code  106574 non-null  object
 8   track_favorites      106574 non-null  int64 
 9   track_genre_top      106574 non-null  object
 10  track_genres         106574 non-null  object
 11  track_genres_all     106574 non-null  object
 12  track_tags           106574 non-null  object
 13  track_title          106574 non-null  object
dtypes: int64(7), object(7)
memory usage: 11.4+ MB


In [152]:
final_tracks.shape

(106574, 14)

### Features

First,  we need to analyse our data. We notice that in this file  there is not any categorical variable. \
Then we decided to apply PCA our numerical data. \

We noticed that some column names contain the same word and are closely related. So we filtered the columns containing those words and applied PCA separately. These columns : **Chroma, mfcc, spectral, tonnetz, zcr**

In [125]:
# First we need to chose numeric and categorical columns 
num_cols = [col for col in features_c.columns if features_c[col].dtypes != "O"]
cat_cols = [col for col in features_c.columns if features_c[col].dtypes == "O"]
cat_cols

[]

In [126]:
[col for col in features_c]

['track_id',
 'chroma_cens_kurtosis_01',
 'chroma_cens_kurtosis_02',
 'chroma_cens_kurtosis_03',
 'chroma_cens_kurtosis_04',
 'chroma_cens_kurtosis_05',
 'chroma_cens_kurtosis_06',
 'chroma_cens_kurtosis_07',
 'chroma_cens_kurtosis_08',
 'chroma_cens_kurtosis_09',
 'chroma_cens_kurtosis_10',
 'chroma_cens_kurtosis_11',
 'chroma_cens_kurtosis_12',
 'chroma_cens_max_01',
 'chroma_cens_max_02',
 'chroma_cens_max_03',
 'chroma_cens_max_04',
 'chroma_cens_max_05',
 'chroma_cens_max_06',
 'chroma_cens_max_07',
 'chroma_cens_max_08',
 'chroma_cens_max_09',
 'chroma_cens_max_10',
 'chroma_cens_max_11',
 'chroma_cens_max_12',
 'chroma_cens_mean_01',
 'chroma_cens_mean_02',
 'chroma_cens_mean_03',
 'chroma_cens_mean_04',
 'chroma_cens_mean_05',
 'chroma_cens_mean_06',
 'chroma_cens_mean_07',
 'chroma_cens_mean_08',
 'chroma_cens_mean_09',
 'chroma_cens_mean_10',
 'chroma_cens_mean_11',
 'chroma_cens_mean_12',
 'chroma_cens_median_01',
 'chroma_cens_median_02',
 'chroma_cens_median_03',
 'chroma_

In [127]:
features_c.head()

,track_id,chroma_cens_kurtosis_01,chroma_cens_kurtosis_02,chroma_cens_kurtosis_03,chroma_cens_kurtosis_04,chroma_cens_kurtosis_05,chroma_cens_kurtosis_06,chroma_cens_kurtosis_07,chroma_cens_kurtosis_08,chroma_cens_kurtosis_09,...,tonnetz_std_04,tonnetz_std_05,tonnetz_std_06,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01
0,2,7.180653,5.230309,0.249321,1.347620,1.482478,0.531371,1.481593,2.691455,0.866868,...,0.054125,0.012226,0.012111,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448
1,3,1.888963,0.760539,0.345297,2.295201,1.654031,0.067592,1.366848,1.054094,0.108103,...,0.063831,0.014212,0.017740,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330
2,5,0.527563,-0.077654,-0.279610,0.685883,1.937570,0.880839,-0.923192,-0.927232,0.666617,...,0.040730,0.012691,0.014759,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861
3,10,3.702245,-0.291193,2.196742,-0.234449,1.367364,0.998411,1.770694,1.604566,0.521217,...,0.074358,0.017952,0.013921,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800
4,20,-0.193837,-0.198527,0.201546,0.258556,0.775204,0.084794,-0.289294,-0.816410,0.043851,...,0.095003,0.022492,0.021355,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993


In [128]:
# Filtering the columns which is related to chroma
filter_chroma_col = [col for col in num_cols if col.startswith('chroma')]
len(filter_chroma_col)

252

In [129]:
# We used PCA for our numerical chroma columns
x2 = features_c.loc[:, filter_chroma_col].values
x2 = StandardScaler().fit_transform(x2)

In [130]:
# Initialize the PCA class by passing the number of components to the constructor.
pca2 = PCA(n_components=32)
# Call the fit and then transform methods by passing the feature set to these methods. 
# The transform method returns the specified number of principal components.
principalComponents = pca2.fit_transform(x2)
# Creating Principal data frame
principalDf_2 = pd.DataFrame(data = principalComponents
             , columns = ["principal component F_chroma" + str(i) for i in range(1,33)])

In [131]:
#pca.explained_variance_ratio_
print ( "Components = ", pca2.n_components_ , ";\nTotal explained variance = ",
      round(pca2.explained_variance_ratio_.sum(),5)  )

Components =  32 ;
Total explained variance =  0.77303


We decided to reduce the number of columns which is related to chroma from 252 to 32. \
The reason why we did not choose it lower than this number (32) is to ensure that the total variance is greater than 70%.

Filtering and PCA to the columns which is related to **mfcc** :

In [132]:
# Filtering the columns which is related to mfcc
filter_mfcc_col = [col for col in num_cols if col.startswith('mfcc')]
len(filter_mfcc_col)

140

In [133]:
# We used PCA for our mfcc columns
x3 = features_c.loc[:, filter_mfcc_col].values
x3 = StandardScaler().fit_transform(x3)

In [134]:
# Initialize the PCA class by passing the number of components to the constructor.
pca3 = PCA(n_components=20)
# Call the fit and then transform methods by passing the feature set to these methods. 
# The transform method returns the specified number of principal components.
principalComponents = pca3.fit_transform(x3)
# Creating Principal data frame
principalDf_3 = pd.DataFrame(data = principalComponents
             , columns = ["principal component F_mfcc " + str(i) for i in range(1,21)])

In [135]:
#pca.explained_variance_ratio_
print ( "Components = ", pca3.n_components_ , ";\nTotal explained variance = ",
      round(pca3.explained_variance_ratio_.sum(),5)  )

Components =  20 ;
Total explained variance =  0.73739


We decided to reduce the number of columns which is related to mfcc from 140 to 21. \
The reason why we did not choose it lower than this number (21) is to ensure that the total variance is greater than 70%.

Filtering and PCA to the columns which is related to **tonnetz** :

In [136]:
# Extracting the columns which is related to tonnetz
filter_tonnetz_col = [col for col in num_cols if col.startswith('tonnetz')]
len(filter_tonnetz_col)

42

In [137]:
# We used PCA for our tonnetz columns
x4 = features_c.loc[:, filter_tonnetz_col].values
x4 = StandardScaler().fit_transform(x4)

In [138]:
# Initialize the PCA class by passing the number of components to the constructor.
pca4 = PCA(n_components=14)
# Call the fit and then transform methods by passing the feature set to these methods. 
# The transform method returns the specified number of principal components.
principalComponents = pca4.fit_transform(x4)
# Creating Principal data frame
principalDf_4 = pd.DataFrame(data = principalComponents
             , columns = ["principal component F_tonnetz " + str(i) for i in range(1,15)])

In [139]:
#pca.explained_variance_ratio_
print ( "Components = ", pca4.n_components_ , ";\nTotal explained variance = ",
      round(pca4.explained_variance_ratio_.sum(),5)  )

Components =  14 ;
Total explained variance =  0.80362


We decided to reduce the number of columns which is related to tonnetz from 42 to 14.
The reason why we did not choose it lower than this number (14) is to ensure that the total variance is greater than 70%.

Filtering and PCA to the columns which is related to **spectral** :

In [140]:
# Extracting the columns which is related to tonnetz
filter_spectral_col = [col for col in num_cols if col.startswith('spectral')]
len(filter_spectral_col)

70

In [141]:
# We used PCA for our spectral columns
x5 = features_c.loc[:, filter_spectral_col].values
x5 = StandardScaler().fit_transform(x5)

In [142]:
# Initialize the PCA class by passing the number of components to the constructor.
pca5 = PCA(n_components=10)
# Call the fit and then transform methods by passing the feature set to these methods. 
# The transform method returns the specified number of principal components.
principalComponents = pca5.fit_transform(x5)
# Creating Principal data frame
principalDf_5 = pd.DataFrame(data = principalComponents
             , columns = ["principal component F_spectral" + str(i) for i in range(1,11)])

In [143]:
#pca.explained_variance_ratio_
print ( "Components = ", pca5.n_components_ , ";\nTotal explained variance = ",
      round(pca5.explained_variance_ratio_.sum(),5))

Components =  10 ;
Total explained variance =  0.70482


We decided to reduce the number of columns which is related to spectral from 70 to 10. \
The reason why we did not choose it lower than this number (10) is to ensure that the total variance is greater than 70%.

Filtering and PCA to the columns which is related to **zcr** :

In [144]:
# Extracting the columns which is related to zcr
filter_zcr_col = [col for col in num_cols if col.startswith('zcr')]
len(filter_zcr_col)

7

Since we only have 7 columns related to zcr, we decided to keep them without applying PCA.

In [145]:
final_features = pd.concat([principalDf_2,principalDf_3,principalDf_4,principalDf_5, features_c[filter_zcr_col],features_c["track_id"]], axis = 1)

In [146]:
final_features.shape

(106574, 84)

In [147]:
final_features

,principal component F_chroma1,principal component F_chroma2,principal component F_chroma3,principal component F_chroma4,principal component F_chroma5,principal component F_chroma6,principal component F_chroma7,principal component F_chroma8,principal component F_chroma9,principal component F_chroma10,...,principal component F_spectral9,principal component F_spectral10,zcr_kurtosis_01,zcr_max_01,zcr_mean_01,zcr_median_01,zcr_min_01,zcr_skew_01,zcr_std_01,track_id
0,0.775576,0.363592,1.361536,-3.699536,-6.147776,-5.360324,-1.897541,2.495669,-0.448805,-2.121177,...,-0.191538,-1.422479,5.758890,0.459473,0.085629,0.071289,0.000000,2.089872,0.061448,2
1,2.396017,-2.216470,-5.514634,-0.510095,1.642135,-3.034920,-3.248693,2.734700,-0.931667,1.088238,...,0.608171,-0.695209,2.824694,0.466309,0.084578,0.063965,0.000000,1.716724,0.069330,3
2,0.238588,-0.583128,-3.281496,-1.874589,-1.639847,-0.529610,-2.849984,1.958701,-0.719224,0.016122,...,-0.264878,-0.586384,6.808415,0.375000,0.053114,0.041504,0.000000,2.193303,0.044861,5
3,6.340997,1.307152,-4.068535,-2.896137,1.793518,-2.828145,-2.905856,1.605868,-0.014288,3.209458,...,-0.826906,-0.567548,21.434212,0.452148,0.077515,0.071777,0.000000,3.542325,0.040800,10
4,-1.902813,-1.745443,-2.876833,2.710265,-0.255265,0.354118,-0.259910,-0.680847,1.649361,3.117164,...,-0.796976,3.432878,16.669037,0.469727,0.047225,0.040039,0.000977,3.189831,0.030993,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106569,1.563061,-1.342362,6.758801,-1.639733,4.023444,2.421768,6.661468,-0.091453,-1.807018,-3.167148,...,1.298608,-0.819860,4.448255,0.172852,0.028773,0.028320,0.003906,0.955388,0.012385,155316
106570,0.976325,-4.937341,2.159991,2.554924,-2.190858,6.301972,-0.158664,1.094713,-0.163789,0.447464,...,1.662814,-0.190155,3.270612,0.196289,0.031116,0.027832,0.002441,1.283060,0.019059,155317
106571,3.719994,-4.143303,2.385700,-1.277101,1.464152,1.783146,3.163201,1.525087,-0.570339,-0.913996,...,1.084437,-0.319742,2.356727,0.212891,0.038450,0.037109,0.003418,0.828569,0.017904,155318
106572,4.565423,-3.903904,0.804833,0.306835,1.538993,-1.027876,0.382183,0.735872,0.115893,0.685599,...,1.487955,-0.617231,6.188604,0.167480,0.041480,0.038086,0.004883,1.818740,0.020133,155319


Once we got all the datasets well filled and standardized, we can start merging them using as key the **track_id** column.

In [153]:
final = final_tracks.merge(final_features.merge(final_echonest, on = "track_id"), on = "track_id")
print(final.shape)
final.head()

(13129, 154)


,track_id,album_id,artist_id,artist_location,track_bit_rate,track_duration,track_listens,track_language_code,track_favorites,track_genre_top,...,ranks_artist_discovery_rank,ranks_artist_familiarity_rank,ranks_artist_hotttnesss_rank,ranks_song_currency_rank,ranks_song_hotttnesss_rank,social_features_artist_discovery,social_features_artist_familiarity,social_features_artist_hotttnesss,social_features_song_currency,social_features_song_hotttnesss
0,2,1,1,New Jersey,256000,168,1293,en,2,Hip-Hop,...,211113.570265,144690.666903,234148.305595,2.919776e+06,2.325537e+06,0.388990,0.386740,0.406370,0.000000,0.000000
1,3,1,1,New Jersey,256000,237,514,en,1,Hip-Hop,...,211113.570265,144690.666903,234148.305595,2.919776e+06,2.325537e+06,0.388990,0.386740,0.406370,0.000000,0.000000
2,5,1,1,New Jersey,256000,206,1151,en,6,Hip-Hop,...,211113.570265,144690.666903,234148.305595,2.919776e+06,2.325537e+06,0.388990,0.386740,0.406370,0.000000,0.000000
3,10,6,6,,192000,161,50135,en,178,Pop,...,2635.000000,2544.000000,397.000000,1.156910e+05,6.760900e+04,0.557339,0.614272,0.798387,0.005158,0.354516
4,134,1,1,New Jersey,256000,207,943,en,3,Hip-Hop,...,211113.570265,144690.666903,234148.305595,2.919776e+06,2.325537e+06,0.388990,0.386740,0.406370,0.000000,0.000000


As we can see, the number of rows is almost 13k, as demanded.